In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install datasets
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-finetuning
%pip install llama-index-readers-file
%pip install llama-index-embeddings-huggingface
%pip install "transformers[torch]"

In [ ]:
import os
os.chdir("/content/drive/MyDrive/ALQAC")
os.listdir()

# Prepare Dataset

In [ ]:
# Load corpus
import json
path = "ALQAC_2025_data/alqac25_train.json"
with open(path, "r") as f:
    train_data = json.load(f)

train_data[:2]

In [ ]:
from pathlib import Path
import json

documents = {}

output_path = Path("./output")
for filename in output_path.glob("*"):
    print(filename.stem)
    with open(filename, "r") as file:
        json_content = json.load(file)
    documents[filename.stem] = json_content



class EmbeddingQAFinetuneDataset(BaseModel):

    Embedding QA Finetuning Dataset.

    Args:
        queries (Dict[str, str]): Dict id -> query.
        corpus (Dict[str, str]): Dict id -> string.
        relevant_docs (Dict[str, List[str]]): Dict query id -> list of doc ids.

In [ ]:
queries = {}
for sample in train_data:
    queries.update({sample['question_id']: sample['text']})

In [ ]:
next(iter(queries.items()))

In [ ]:
# id = "article_id + document_id"
corpus = {}
for document in documents:
    document_id = document
    for article in documents[document_id]:
        corpus_id = f"{document_id} - {list(article.keys())[0]}"
        corpus.update({corpus_id: next(iter(article.values()))})
next(iter(corpus.items()))

In [ ]:
relevant_docs = {}

for sample in train_data:
    question_id = sample["question_id"]
    relevant_articles = []
    for article in sample["relevant_articles"]:
        relevant_articles.append(f"{article['law_id']} - {article['article_id']}")
    relevant_docs.update({question_id: relevant_articles})
next(iter(relevant_docs.items()))

# Load Dataset

In [ ]:
from datasets import Dataset

anchors = []
positives = []
# Open a file, perform preprocessing, filtering, cleaning, etc.
# and append to the lists

dataset = Dataset.from_dict({
    "anchor": queries.values,
    "positive": positives,
})

# Run embedding finetuning